# Prepairing chesscom data to use

## Import and functions

In [3]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

In [4]:
# convert unixtime to time:
# fr_unixtime(1565211491) -> '2019-08-07 20:58:11'
def fr_unixtime(ts):
    from datetime import datetime
    return datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S')

# convert integer month number to string API format:
# 1 -> '01' 
# 9 -> '09' 
# 12 > '12'
# 112 -> -1
def get_number(x):
    if x>=10 and x<100:
        return str(x)
    elif x<10:
        return '0'+str(x)
    else:
        return -1
    
# retunr classoc Elo propabilities
# elo_prob(2882, 2722) -> 0.7152 (72% chanses Carlsen (2882) to beat Wan Hao (2722))
def elo_prob(rw, rb):
    return 1/(1+np.power(10, (rb-rw)/400))


# working with pgn
# pip install pgn-parser
def get_pgn(text):
    from pgn_parser import pgn, parser
    game = parser.parse(text, actions=pgn.Actions())
    
    try:
        score=game.tag_pairs['Result']
    except:
        score='Unknown'  
    try:
        date=game.tag_pairs['Date']
    except:
        date='Unknown'
    try:
        time=game.tag_pairs['UTCTime']
    except:
        time='Unknown'
    try:
        eco=game.tag_pairs['ECO']
    except:
        eco='Unknown'
    try:
        ecourl=game.tag_pairs['ECOUrl']
    except:
        ecourl='Unknown'
    return {'score':score, 
            'date': date, 
            'time':time, 
            'ECO':eco, 
            'ECO_url':ecourl}

# functions for predictions
# find col with target user (white or black)
def find_col(target_user, col1, wh_val, bl_val):
    if col1==target_user:
        return wh_val
    else:
        return bl_val
# reverse fun of find_col    
def find_opp(target_user, col1, wh_val, bl_val):
    if col1!=target_user:
        return wh_val
    else:
        return bl_val


In [5]:
# get json form site and return string
# exmple get_api_data_to_str('eric', '2014', '01')
def get_api_data_to_str(player, year, month):
    import requests
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
    if response.status_code ==429:
        with open('data/429.txt', 'w+') as f:
            f.write('variant, ')
            f.write(st)
            f.close()
    return st

# convert api data to pandas for next working
def get_str_data_to_pandas(strng):
    import json
    js = json.loads(strng)
    df=pd.DataFrame(js)
    
    return df

# convert multilevel json to table
def prepair_pandas_multilevel_data(df):
    num=len(df)
    new_df=pd.DataFrame()
    for i in range(num):
        b=pd.io.json.json_normalize(df.values[i])
        new_df=pd.concat([new_df, b])
        
    return new_df 

# get player games in month with preparation
def get_pl_stat(game_stat):    
    # chess result dictionary
    res_dict={
    'win': 1.0,
    'checkmated': 0.0,
    'agreed': 0.5,
    'repetition': 0.5,
    'timeout': 0.0,
    'resigned': 0.0,
    'stalemate': 0.5,
    'lose': 0.0,
    'insufficient': 0.5,
    '50move': 0.5,
    'abandoned': 0.0,
    'kingofthehill': 0.5,
    'threecheck': 0.5,
    'timevsinsufficient': 0.5,
    'bughousepartnerlose': 0.0
    }

    # number of games
    num=len(game_stat)
    
    # cut long and unusefull columns
    game_stat=game_stat[['rules', 'time_class', 'time_control', 'rated',
        'white.@id', 'white.rating', 'white.result', 'white.username',
         'black.@id', 'black.rating', 'black.result', 'black.username',
         'end_time', 'pgn', 
         'url']]
    # score 0.0, 0.5 or 1.0
    game_stat['white.score']=game_stat['white.result'].map(res_dict)
    game_stat['black.score']=game_stat['black.result'].map(res_dict)
    
    # pgn - long string
    game_stat['pgn']=game_stat['pgn'].apply(get_pgn)
    # result - string like '1-0'
    game_stat['result']=game_stat['pgn'].apply(lambda x: x['score'])
    
    game_stat['date']=game_stat['pgn'].apply(lambda x: x['date'])
    game_stat['time']=game_stat['pgn'].apply(lambda x: x['time'])
    
    # ECO - string like 'B10'
    game_stat['eco']=game_stat['pgn'].apply(lambda x: x['ECO'])
    # ECOurl - url like 'https://www.chess.com/openings/B10-Caro-Kann-Defense-2.Nf3-d5'
    game_stat['eco_url']=game_stat['pgn'].apply(lambda x: x['ECO_url'])
    
    # propability from classic formula
    game_stat['white_elo_forecast']=game_stat[['white.rating', 'black.rating']].apply(lambda x: elo_prob(*x), axis=1)
    game_stat['black_elo_forecast']=game_stat[['black.rating', 'white.rating']].apply(lambda x: elo_prob(*x), axis=1)
    
    # after it np.sum(df['game']) means number of games
    game_stat['game']=1
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
    # period - stirng like '2019-08'
    game_stat['period']=game_stat['date'].dt.year.astype('str')+'-'+game_stat['date'].dt.month.astype('str')
        
    game_stat.columns=['rules', 'time_class', 'time_control', 'rated', 'white_url',
       'white.rating', 'white.result', 'white.username', 'black_url',
       'black.rating', 'black.result', 'black.username', 'end_time', 'pgn', 'game_url',
       'white.score', 'black.score', 'result', 'date', 'time', 'eco',
       'eco_url', 'white_elo_forecast', 'black_elo_forecast',
       'game', 'period']
    game_stat=game_stat[[
        'rules', 'time_class', 'time_control', 'rated', 'game',
        'result', 'date', 'time', 'period',
        'white.username', 'black.username',
        'white.rating', 'black.rating', 
        'white.result', 'black.result', 
        'white.score', 'black.score', 
        'white_elo_forecast', 'black_elo_forecast',
        'eco', 'eco_url', 
        'game_url', 'white_url', 'black_url'
    ]]
    return game_stat
    


In [6]:
# main function
def get_btch(player, year, month):
    import datetime
    
    # API -> string
    
    delay_lst=[0, 1, 4]
    # we give 3 attemps to get data
    for cs in range(3):
        try:
            data_string=get_api_data_to_str(player, year, month)
        except Exception:
            print('Error: appempt num', cs)
            # wait 0, 1 or 2 seconds after next attempt
            time.sleep(delay_lst[cs])
            if cs==2:
                print('No load data')
    
    # string -> json -> DataFrame
    multidata=get_str_data_to_pandas(data_string)
    
    # DataFrame -> use json normalise -> good df
    usefull_data=prepair_pandas_multilevel_data(multidata)
    
    # game prepair
    workng_data=get_pl_stat(usefull_data)
    
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    workng_data.to_csv('source/'+player+'_'+year+'_'+month+'_'+now+'.csv')
    
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(now, ': btch '+player+'_'+year+'_'+month+' finished')

In [7]:
# start getting with threading magic
def user_parse(user_list, year_range):   
    import threading
    # status counter
    i=0
    for user in user_list:
        for year in year_range:
            for month in range(1,13):
                t = threading.Thread(target=get_btch, args=(user, str(year), get_number(month)))
                t.start()
                i=i+1
    print('All btch', i,  'are started')   

In [8]:
# read all json in folder and concate DataFrame
def read_files(path):
    # reading
    l=[]
    cntr=0
    import os
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".csv"):
                 l.append(os.path.join(root, file))
            cntr=cntr+1
    print('Total', cntr, 'files founded')

    # merging
    i=0
    df=pd.DataFrame()
    for link in l:
        try:
            dfb=pd.read_csv(link)
        except:
            dfb=pd.DataFrame()
        df=pd.concat([df, dfb])
        
    df=df.drop_duplicates()
    return df

In [36]:
# data for predict current user
def learn_prepair(df, target_user):
    df=df[(df['white.username']==target_user)|(df['black.username']==target_user)]
    df['target_user']=target_user
    df['score']=df[['target_user', 'white.username', 'white.score', 'black.score']].apply(lambda x: find_col(*x), axis=1)
    df['rating']=df[['target_user','white.username', 'white.rating', 'black.rating']].apply(lambda x: find_col(*x), axis=1)
    df['opponent_rating']=df[['target_user', 'white.username', 'white.rating', 'black.rating']].apply(lambda x: find_opp(*x), axis=1)
    df['base_elo_forec']=df[['target_user', 'white.username', 'white_elo_forecast', 'black_elo_forecast']].apply(lambda x: find_col(*x), axis=1)
    df['color']=np.where(df['white.username']==target_user, 'w', 'b')
    
    df['date'] = df['date'].astype('datetime64[ns]')
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    
    return df

In [23]:
# get players list by countries
def get_county_players_list(country):
    import requests
    response = requests.get('https://api.chess.com/pub/country/'+country+'/players')
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
        
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
 
    try:
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write(st)
        file.close()
    except Exception:
        error_st=response.status_code+st[0:3000]
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write( error_st)
        file.close()


## Start working

In [11]:
# collect players by countries
# countries_list=['RU']
# get_county_players_list(countries_list[0])

# user_list=pd.read_json('player_lists/players_list_RU_2019-08-11 19:55:21.json').sample(50)['players'].values
user_list=['andreyvict', 'sever043', 'Rosolimo']

In [12]:
user_list[0:30]

['andreyvict', 'sever043', 'Rosolimo']

In [17]:
year_range=range(2012, 2020)
#user_parse(user_list, year_range)

In [37]:
%%time
df=read_files('source/')

Total 551 files founded
CPU times: user 47.9 s, sys: 407 ms, total: 48.3 s
Wall time: 52.9 s


In [38]:
len(df)

56873

In [39]:
df.sample(6)

,Unnamed: 0,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url
430,0,chess,blitz,180,True,1,0-1,2019-04-24,17:28:43,2019-4,GillRS,oneelevenchemp,1187,1244,resigned,win,0.0,1.0,0.418699,0.581301,B21,https://www.chess.com/openings/B21-Sicilian-Defense-McDonnell-Attack-2...Nc6-3.Nf3-e6,https://www.chess.com/live/game/3643977210,https://api.chess.com/pub/player/gillrs,https://api.chess.com/pub/player/oneelevenchemp
273,0,chess,blitz,300+5,True,1,0-1,2019-07-23,17:01:59,2019-7,deputat1,chesstechman,1312,1377,resigned,win,0.0,1.0,0.407534,0.592466,B01,https://www.chess.com/openings/B01-Scandinavian-Defense-Modern-Variation,https://www.chess.com/live/game/3887971690,https://api.chess.com/pub/player/deputat1,https://api.chess.com/pub/player/chesstechman
74,0,chess,blitz,300,True,1,0-1,2015-02-16,20:21:42,2015-2,Rosolimo,arantxa90,1596,1596,checkmated,win,0.0,1.0,0.500000,0.500000,E00,https://www.chess.com/openings/E00-Indian-Game-East-Indian-Defense,https://www.chess.com/live/game/1061920533,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/arantxa90
49,0,chess,blitz,600,True,1,0-1,2018-03-24,15:20:52,2018-3,xxxmarios,oneelevenchemp,920,955,checkmated,win,0.0,1.0,0.449801,0.550199,B07,https://www.chess.com/openings/B07-Pirc-Defense,https://www.chess.com/live/game/2698516881,https://api.chess.com/pub/player/xxxmarios,https://api.chess.com/pub/player/oneelevenchemp
228,0,chess,blitz,300+5,True,1,1-0,2019-07-21,11:49:30,2019-7,neshandvie,deputat1,1328,1360,win,checkmated,1.0,0.0,0.454078,0.545922,B20,https://www.chess.com/openings/B20-Sicilian-Defense-Wing-Marshall-Carlsbad-Variation,https://www.chess.com/live/game/3881705250,https://api.chess.com/pub/player/neshandvie,https://api.chess.com/pub/player/deputat1
19,0,chess,rapid,1800,True,1,0-1,2019-06-05,18:48:38,2019-6,oles,temrkan1,1324,1370,resigned,win,0.0,1.0,0.434185,0.565815,B21,https://www.chess.com/openings/B21-Sicilian-Defense-Smith-Morra-Gambit-2...cxd4-3.Qxd4-Nc6-4.Qd1,https://www.chess.com/live/game/3758594446,https://api.chess.com/pub/player/oles,https://api.chess.com/pub/player/temrkan1


In [ ]:
ds=learn_prepair(df, 'Rosolimo')
#ds=learn_prepair(df, 'sever043')

In [16]:
ds.sample(3)

,Unnamed: 0,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url,target_user,score,rating,opponent_rating,base_elo_forec,color
200,0,chess,blitz,300,True,1,1-0,2014-03-07,06:53:20,2014-3,rookie_12,fearplay,1234,1332,win,resigned,1.0,0.0,0.362592,0.637408,D20,https://www.chess.com/openings/D20-Queens-Gambit-Accepted-Old-Variation,https://www.chess.com/live/game/745201374,https://api.chess.com/pub/player/rookie_12,https://api.chess.com/pub/player/fearplay,Rosolimo,0.0,1332,1234,0.637408,b
43,0,chess,blitz,300+5,True,1,1-0,2017-03-07,19:48:35,2017-3,tomazio,DV007,1544,1522,win,timeout,1.0,0.0,0.531618,0.468382,B07,https://www.chess.com/openings/B07-Pirc-Defense-Antal-Defense,https://www.chess.com/live/game/1985847956,https://api.chess.com/pub/player/tomazio,https://api.chess.com/pub/player/dv007,Rosolimo,0.0,1522,1544,0.468382,b
281,0,chess,blitz,600,True,1,0-1,2015-09-26,10:19:09,2015-9,nitikarai,madrid7676,942,927,resigned,win,0.0,1.0,0.521573,0.478427,C20,https://www.chess.com/openings/C20-Kings-Pawn-Opening-Wayward-Queen-Attack-2...Nc6,https://www.chess.com/live/game/1293447111,https://api.chess.com/pub/player/nitikarai,https://api.chess.com/pub/player/madrid7676,Rosolimo,1.0,927,942,0.478427,b


In [27]:
ds.groupby(['year', 'month']).sum()[['game', 'score']]#.plot()

game  score
year month             
2012 9        95   55.0
2013 9       215  105.5
     10      284  137.5
     11      287  146.0
     12      384  183.5
2014 1       355  182.5
     2       294  132.5
     3       262  127.0
     4       173   85.5
     5       202   89.5
     6       182   89.0
     7       222  112.0
     8       308  151.0
     9       149   70.5
     10       88   48.0
     11      193   88.0
     12      209  103.0
2015 1       147   72.0
     2       168   80.0
     3       189   96.0
     4       191   91.0
     5       222  106.0
     6        99   52.5
     7       349  172.5
     8       192   89.0
     9       258  119.0
     10      181   93.0
     11      185   87.0
     12      250  126.0
2016 1       247  117.0
     2       234  114.5
     3       109   49.0
     4       136   71.0
     5       130   62.5
     6        10    4.0
     7       113   58.0
     8       183   92.0
     9       114   50.5
     10      207  104.0
     11      116   52.0
     12      186   96.5
2017 1       158   82.0
     2        62   34.5
     3        94   41.5
     4       201  102.5
     5        56   24.5
     6        55   27.5
     7       124   66.0
     8       164   80.5
     9       140   77.0
     10      145   71.0
     11       71   40.5
     12       69   35.5
2018 1        98   41.5
     2        66   36.0
     3       146   78.5
     4       142   73.5
     5       219  112.0
     6       220  112.0
     7       183   85.5
     8       181   92.5
     9       266  128.0
     10      271  133.5
     11      332  167.0
     12      388  186.0
2019 1       226  103.5
     2       196   92.5
     3       330  169.0
     4       160   78.0
     5       112   57.5
     6       112   52.5
     7       151   73.5
     8        62   34.0

In [18]:
# tst=get_api_data_to_str('Rosolimo', '2015', '11')